In [2]:
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import argparse
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout,GlobalMaxPool2D,BatchNormalization,Dropout
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight

2024-04-29 07:24:17.641340: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-29 07:24:17.641395: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-29 07:24:17.642819: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
nrem_sample = np.load('/kaggle/input/sleep-state-classification/nrem_sample.npy')
wakefulness_sample =  np.load('/kaggle/input/sleep-state-classification/wakefulness_sample.npy')
rem_sample = np.load('/kaggle/input/sleep-state-classification/rem_sample.npy')

In [ ]:
nrem_sample = np.load('/kaggle/input/sleep-state-classification/nrem_sample.npy')
wakefulness_sample =  np.load('/kaggle/input/sleep-state-classification/wakefulness_sample.npy')
rem_sample 

# Loss Function

In [ ]:
def focal_loss(gamma=3.0, alpha=0.3):
    def focal_loss_fixed(y_true, y_pred):
        epsilon = 1e-9
        y_pred = tf.clip_by_value(y_pred, epsilon, 1.0 - epsilon)

        # Compute cross entropy
        cross_entropy = -y_true * tf.math.log(y_pred)

        # Compute focal loss
        focal_loss = alpha * tf.pow(1 - y_pred, gamma) * cross_entropy

        # Sum over classes
        return tf.reduce_sum(focal_loss, axis=-1)
    
    return focal_loss_fixed

# 2D CNN algorithm

In [ ]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


# Create labels: Wakefulness=0, NREM=1, REM=2
X = np.concatenate([wakefulness_sample, nrem_sample, rem_sample])
y = np.array([0]*len(wakefulness_sample) + [1]*len(nrem_sample) + [2]*len(rem_sample))

# Normalize data
X = X.astype('float32') / 255.0  # Adjust based on your actual data range

# One-hot encode labels
y = to_categorical(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

# Compute class weights for balancing the data
class_weights = compute_class_weight('balanced', classes=np.unique(np.argmax(y, axis=1)), y=np.argmax(y, axis=1))
class_weight_dict = dict(enumerate(class_weights))

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss=focal_loss(), metrics=['accuracy'])

In [ ]:
# Train the model with class weights
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), class_weight=class_weight_dict)

In [ ]:
# Evaluate model performance
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc * 100:.2f}%')

In [ ]:
# Predict the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert probabilities to class labels
y_true_classes = np.argmax(y_test, axis=1)  # Convert one-hot encoded labels to class labels

# Compute the confusion matrix
cm = confusion_matrix(y_true_classes, y_pred_classes)

# Compute the accuracy
accuracy = accuracy_score(y_true_classes, y_pred_classes)

# Print the classification report
print("Classification Report:\n", classification_report(y_true_classes, y_pred_classes))

# Displaying the confusion matrix using seaborn
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['NREM', 'REM', 'Wakefulness'], yticklabels=['NREM', 'REM', 'Wakefulness'])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

# Print accuracy
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# VGG19 algorithm

In [ ]:
# Create labels
nrem_labels = np.zeros(len(nrem_sample))
rem_labels = np.ones(len(rem_sample))
wakefulness_labels = np.full(len(wakefulness_sample), 2)

# Combine data
X = np.concatenate((nrem_sample, rem_sample, wakefulness_sample), axis=0)
y = np.concatenate((nrem_labels, rem_labels, wakefulness_labels), axis=0)

# One-hot encode labels
y = to_categorical(y, num_classes=3)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Input


def build_custom_vgg19(input_shape=(36, 168, 168)):
    # Create a VGG-like model for 36 input channels
    input_tensor = Input(shape=input_shape)
    x = BatchNormalization()(input_tensor)  # Normalize the input
    # Add VGG19 layers, initialized without weights
    base_model = VGG19(include_top=False, weights=None, input_tensor=x)

    # Customize the training of the initial layers or leave them trainable
    for layer in base_model.layers[:17]:
        layer.trainable = False

    x = base_model.output
    x = GlobalMaxPool2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(3, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    return model

# Instantiate and compile the model
model = build_custom_vgg19()
model.compile(optimizer=Adam(learning_rate=1e-4), loss=focal_loss(), metrics=['accuracy'])

# Assuming X_train, y_train, X_test, y_test are correctly shaped and split
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.1)

In [ ]:
# Predict the test set
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert probabilities to class labels
y_true_classes = np.argmax(y_test, axis=1)  # Convert one-hot encoded labels to class labels

# Compute the confusion matrix
cm = confusion_matrix(y_true_classes, y_pred_classes)

# Compute the accuracy
accuracy = accuracy_score(y_true_classes, y_pred_classes)

# Print the classification report
print("Classification Report:\n", classification_report(y_true_classes, y_pred_classes))

# Displaying the confusion matrix using seaborn
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['NREM', 'REM', 'Wakefulness'], yticklabels=['NREM', 'REM', 'Wakefulness'])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

# Print accuracy
print(f"Test Accuracy: {accuracy * 100:.2f}%")
